# Close Base Station Extraction

In this notebook, we aim to find closest Base Station for each given Base Station, so that
if we have a new Base Station product with no prior information about, we can go and check the closest 
Base Station with previous information to it.

By closeness, we mean different things depending on the KDTree implementations below:
- 1: Close BS depending on the cell-level static features
- 2: Close BS depending on the cell-level static features and dynamic feature profile (mean and std of the dynamic features)

Then, we use this information in the further training process in Neural Network notebook.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KDTree
import re

pd.set_option('display.max_columns', 500)

np.random.seed(101)

## KDTree 1

Find close BS data only using the static columns (TXPower, Frequency, Bandwidth, RUType, etc.)

In [2]:
df_info = pd.read_csv("data/base_station_basic_info.csv")
scaled_cols = ['Mode', 'Frequency', 'Bandwidth','Antennas', 'TXpower']

df_info["Mode"] = df_info["Mode"].map({"Mode1": 0, "Mode2" : 1})
df_info["RUType"] = df_info["RUType"].map({t: int(re.findall(r'\d+',t)[0]) for t in sorted(df_info["RUType"].unique())})

mscaler = MinMaxScaler()
df_info[scaled_cols] = mscaler.fit_transform(df_info[scaled_cols])
df_info.head(3)

,BS,CellName,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower
0,B_0,Cell0,1,1.0,0.254004,1.0,0.047619,0.499226
1,B_1,Cell0,2,1.0,0.456576,1.0,0.047619,0.499226
2,B_2,Cell0,1,1.0,0.254004,1.0,0.047619,0.499226


In [3]:
tree = KDTree(df_info[["RUType","Bandwidth","Antennas","TXpower"]])

k=250
dist, ind = tree.query(df_info[["RUType","Bandwidth","Antennas","TXpower"]], k=k)

ivec = [f"I{i}" for i in range(k)]
dvec = [f"D{i}" for i in range(k)]

df_kdtree = pd.DataFrame(np.hstack((ind[:,:],dist[:,:])),columns=[ivec+dvec])
df_kdtree[ivec] = df_kdtree[ivec].astype(np.int32)

In [4]:
# These are cell-level closeness data, so there are more than 1020 (number of distinct BSs) records
df_kdtree

,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,I14,I15,I16,I17,I18,I19,I20,I21,I22,I23,I24,I25,I26,I27,I28,I29,I30,I31,I32,I33,I34,I35,I36,I37,I38,I39,I40,I41,I42,I43,I44,I45,I46,I47,I48,I49,I50,I51,I52,I53,I54,I55,I56,I57,I58,I59,I60,I61,I62,I63,I64,I65,I66,I67,I68,I69,I70,I71,I72,I73,I74,I75,I76,I77,I78,I79,I80,I81,I82,I83,I84,I85,I86,I87,I88,I89,I90,I91,I92,I93,I94,I95,I96,I97,I98,I99,I100,I101,I102,I103,I104,I105,I106,I107,I108,I109,I110,I111,I112,I113,I114,I115,I116,I117,I118,I119,I120,I121,I122,I123,I124,I125,I126,I127,I128,I129,I130,I131,I132,I133,I134,I135,I136,I137,I138,I139,I140,I141,I142,I143,I144,I145,I146,I147,I148,I149,I150,I151,I152,I153,I154,I155,I156,I157,I158,I159,I160,I161,I162,I163,I164,I165,I166,I167,I168,I169,I170,I171,I172,I173,I174,I175,I176,I177,I178,I179,I180,I181,I182,I183,I184,I185,I186,I187,I188,I189,I190,I191,I192,I193,I194,I195,I196,I197,I198,I199,I200,I201,I202,I203,I204,I205,I206,I207,I208,I209,I210,I211,I212,I213,I214,I215,I216,I217,I218,I219,I220,I221,I222,I223,I224,I225,I226,I227,I228,I229,I230,I231,I232,I233,I234,I235,I236,I237,I238,I239,I240,I241,I242,I243,I244,I245,I246,I247,I248,I249,D0,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D20,D21,D22,D23,D24,D25,D26,D27,D28,D29,D30,D31,D32,D33,D34,D35,D36,D37,D38,D39,D40,D41,D42,D43,D44,D45,D46,D47,D48,D49,D50,D51,D52,D53,D54,D55,D56,D57,D58,D59,D60,D61,D62,D63,D64,D65,D66,D67,D68,D69,D70,D71,D72,D73,D74,D75,D76,D77,D78,D79,D80,D81,D82,D83,D84,D85,D86,D87,D88,D89,D90,D91,D92,D93,D94,D95,D96,D97,D98,D99,D100,D101,D102,D103,D104,D105,D106,D107,D108,D109,D110,D111,D112,D113,D114,D115,D116,D117,D118,D119,D120,D121,D122,D123,D124,D125,D126,D127,D128,D129,D130,D131,D132,D133,D134,D135,D136,D137,D138,D139,D140,D141,D142,D143,D144,D145,D146,D147,D148,D149,D150,D151,D152,D153,D154,D155,D156,D157,D158,D159,D160,D161,D162,D163,D164,D165,D166,D167,D168,D169,D170,D171,D172,D173,D174,D175,D176,D177,D178,D179,D180,D181,D182,D183,D184,D185,D186,D187,D188,D189,D190,D191,D192,D193,D194,D195,D196,D197,D198,D199,D200,D201,D202,D203,D204,D205,D206,D207,D208,D209,D210,D211,D212,D213,D214,D215,D216,D217,D218,D219,D220,D221,D222,D223,D224,D225,D226,D227,D228,D229,D230,D231,D232,D233,D234,D235,D236,D237,D238,D239,D240,D241,D242,D243,D244,D245,D246,D247,D248,D249
0,502,430,742,532,534,563,539,507,7,0,63,557,2,62,77,79,555,113,112,172,174,301,188,386,209,180,303,392,436,500,428,710,643,573,773,658,735,700,517,19,595,694,758,775,692,540,182,814,1045,330,366,439,485,709,718,628,729,603,637,766,526,656,26,541,812,789,28,8,35,17,46,89,105,607,680,806,688,127,229,224,170,181,241,297,236,190,191,252,262,223,324,354,362,701,437,474,477,626,820,712,25,605,727,728,734,572,519,525,635,634,128,745,795,760,781,10,43,41,36,16,34,719,91,97,664,665,667,673,682,510,597,1193,1205,1204,1203,721,508,106,108,250,151,150,142,144,135,152,243,269,270,298,231,234,168,260,357,258,160,162,322,346,348,468,355,382,445,476,475,466,478,249,516,235,388,118,133,111,794,120,141,126,703,804,805,1094,1157,1095,803,1,189,389,304,4,306,536,210,387,533,535,743,81,3,80,50,559,1190,798,114,558,115,543,299,556,367,528,542,606,797,799,740,300,1069,1146,1037,1038,1214,1215,1216,1147,1070,1068,1213,1202,1201,1200,1148,527,1158,933,1039,1110,1020,1097,1021,1022,1040,1049,1112,1050,1109,1111,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.031746,0.

In [5]:
df_kdtree.iloc[:1020][ivec].to_csv("data/processed/close_bs_data_only_static.csv",index=False) # Use only cell0 data to measure closeness, discard the rest

## KDTree 2

In [6]:
df_data1 = pd.read_csv("data/processed/multi_cell_training_base.csv")
df_data2 = pd.read_csv("data/processed/multi_cell_prediction_base.csv")
df_data = pd.concat((df_data1,df_data2),axis=0)
df_data = df_data.fillna(0)

scaled_column_names = ["load",	"ESMode1",	"ESMode2",	"ESMode6",	"Frequency", "Bandwidth",	"TXpower"]
scaled_columns = []
for i in range(4): scaled_columns += list(pd.Series(scaled_column_names)+ f"_{i}")
scaled_columns += ["Antennas"]
mscaler = MinMaxScaler()

cluster_column_names = ["load",	"ESMode1",	"ESMode2",	"ESMode6",	"Frequency",	"Bandwidth",	"TXpower", 'Type10', 'Type11', 'Type12', 'Type2', 'Type3', 'Type4', 'Type5', 'Type6', 'Type7', 'Type8', 'Type9']
cluster_cols = []
for i in range(4): cluster_cols += list(pd.Series(cluster_column_names)+ f"_{i}")

important_column_names =  ["load", 'Type10', 'Type11', 'Type12', 'Type2', 'Type3', 'Type4', 'Type5', 'Type6', 'Type7', 'Type8', 'Type9']
important_cols = []
for i in range(4): important_cols += list(pd.Series(important_column_names)+ f"_{i}")

In [7]:
cols = ["BS"]+cluster_cols

df_mean = df_data[cols].groupby("BS").mean().reset_index()
df_mean[important_cols] = df_mean[important_cols]*5
df_std = df_data[cols].groupby("BS").std().reset_index()
df_query = pd.merge(df_mean,df_std,on="BS", suffixes=(".mean",".std"))
df_query = df_query.fillna(0)

In [8]:
df_query.index = df_query["BS"].str.split("_").apply(lambda x:x[1]).astype(int)
df_query = df_query.sort_index()
df_query

,BS,load_0.mean,ESMode1_0.mean,ESMode2_0.mean,ESMode6_0.mean,Frequency_0.mean,Bandwidth_0.mean,TXpower_0.mean,Type10_0.mean,Type11_0.mean,Type12_0.mean,Type2_0.mean,Type3_0.mean,Type4_0.mean,Type5_0.mean,Type6_0.mean,Type7_0.mean,Type8_0.mean,Type9_0.mean,load_1.mean,ESMode1_1.mean,ESMode2_1.mean,ESMode6_1.mean,Frequency_1.mean,Bandwidth_1.mean,TXpower_1.mean,Type10_1.mean,Type11_1.mean,Type12_1.mean,Type2_1.mean,Type3_1.mean,Type4_1.mean,Type5_1.mean,Type6_1.mean,Type7_1.mean,Type8_1.mean,Type9_1.mean,load_2.mean,ESMode1_2.mean,ESMode2_2.mean,ESMode6_2.mean,Frequency_2.mean,Bandwidth_2.mean,TXpower_2.mean,Type10_2.mean,Type11_2.mean,Type12_2.mean,Type2_2.mean,Type3_2.mean,Type4_2.mean,Type5_2.mean,Type6_2.mean,Type7_2.mean,Type8_2.mean,Type9_2.mean,load_3.mean,ESMode1_3.mean,ESMode2_3.mean,ESMode6_3.mean,Frequency_3.mean,Bandwidth_3.mean,TXpower_3.mean,Type10_3.mean,Type11_3.mean,Type12_3.mean,Type2_3.mean,Type3_3.mean,Type4_3.mean,Type5_3.mean,Type6_3.mean,Type7_3.mean,Type8_3.mean,Type9_3.mean,load_0.std,ESMode1_0.std,ESMode2_0.std,ESMode6_0.std,Frequency_0.std,Bandwidth_0.std,TXpower_0.std,Type10_0.std,Type11_0.std,Type12_0.std,Type2_0.std,Type3_0.std,Type4_0.std,Type5_0.std,Type6_0.std,Type7_0.std,Type8_0.std,Type9_0.std,load_1.std,ESMode1_1.std,ESMode2_1.std,ESMode6_1.std,Frequency_1.std,Bandwidth_1.std,TXpower_1.std,Type10_1.std,Type11_1.std,Type12_1.std,Type2_1.std,Type3_1.std,Type4_1.std,Type5_1.std,Type6_1.std,Type7_1.std,Type8_1.std,Type9_1.std,load_2.std,ESMode1_2.std,ESMode2_2.std,ESMode6_2.std,Frequency_2.std,Bandwidth_2.std,TXpower_2.std,Type10_2.std,Type11_2.std,Type12_2.std,Type2_2.std,Type3_2.std,Type4_2.std,Type5_2.std,Type6_2.std,Type7_2.std,Type8_2.std,Type9_2.std,load_3.std,ESMode1_3.std,ESMode2_3.std,ESMode6_3.std,Frequency_3.std,Bandwidth_3.std,TXpower_3.std,Type10_3.std,Type11_3.std,Type12_3.std,Type2_3.std,Type3_3.std,Type4_3.std,Type5_3.std,Type6_3.std,Type7_3.std,Type8_3.std,Type9_3.std
BS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,B_0,2.709666,0.000000,0.0,0.000000,365.000,20.0,6.875934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.150069,0.0,0.0,0.0,77.632727,0.363636,1.332273,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.213869,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.070198,0.0,0.0,0.0,165.311253,0.774328,2.836945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,B_1,0.192854,0.000000,0.0,0.000000,532.000,20.0,6.875934,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022433,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,B_2,0.928543,0.000000,0.0,0.000000,365.000,20.0,6.875934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.064543,0.0,0.0,0.0,77.632727,0.363636,1.332273,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.079548,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028351,0.0,0.0,0.0,165.311253,0.774328,2.836945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [9]:
tree2 = KDTree(df_query.drop(columns="BS"))

k=250
dist, ind = tree2.query(df_query.drop(columns="BS"), k=k)

ivec = [f"I{i}" for i in range(k)]
dvec = [f"D{i}" for i in range(k)]

df_kdtree2 = pd.DataFrame(data=ind,columns=ivec)
df_kdtree2[ivec] = df_kdtree2[ivec].astype(np.int32)

In [10]:
df_kdtree2

,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,I14,I15,I16,I17,I18,I19,I20,I21,I22,I23,I24,I25,I26,I27,I28,I29,I30,I31,I32,I33,I34,I35,I36,I37,I38,I39,I40,I41,I42,I43,I44,I45,I46,I47,I48,I49,I50,I51,I52,I53,I54,I55,I56,I57,I58,I59,I60,I61,I62,I63,I64,I65,I66,I67,I68,I69,I70,I71,I72,I73,I74,I75,I76,I77,I78,I79,I80,I81,I82,I83,I84,I85,I86,I87,I88,I89,I90,I91,I92,I93,I94,I95,I96,I97,I98,I99,I100,I101,I102,I103,I104,I105,I106,I107,I108,I109,I110,I111,I112,I113,I114,I115,I116,I117,I118,I119,I120,I121,I122,I123,I124,I125,I126,I127,I128,I129,I130,I131,I132,I133,I134,I135,I136,I137,I138,I139,I140,I141,I142,I143,I144,I145,I146,I147,I148,I149,I150,I151,I152,I153,I154,I155,I156,I157,I158,I159,I160,I161,I162,I163,I164,I165,I166,I167,I168,I169,I170,I171,I172,I173,I174,I175,I176,I177,I178,I179,I180,I181,I182,I183,I184,I185,I186,I187,I188,I189,I190,I191,I192,I193,I194,I195,I196,I197,I198,I199,I200,I201,I202,I203,I204,I205,I206,I207,I208,I209,I210,I211,I212,I213,I214,I215,I216,I217,I218,I219,I220,I221,I222,I223,I224,I225,I226,I227,I228,I229,I230,I231,I232,I233,I234,I235,I236,I237,I238,I239,I240,I241,I242,I243,I244,I245,I246,I247,I248,I249
0,0,190,700,694,191,692,758,445,2,760,572,151,437,439,680,152,112,436,682,688,573,113,428,7,766,223,236,235,118,430,224,126,120,476,355,474,357,721,362,656,475,664,181,468,477,466,709,478,182,727,719,658,703,97,673,701,665,386,392,667,91,301,303,128,89,127,806,366,539,532,534,388,77,231,637,229,180,643,174,41,635,734,46,234,735,36,540,172,79,35,742,43,541,557,814,17,820,19,135,710,34,812,555,250,28,382,133,26,718,141,25,252,563,188,258,712,517,525,241,170,243,519,249,526,795,348,789,794,354,209,607,346,324,322,106,603,108,330,595,597,111,16,8,10,485,773,634,626,510,781,775,150,516,508,142,62,628,144,63,500,502,507,269,262,270,260,728,168,162,160,729,85,394,332,769,57,459,47,649,407,689,316,580,198,554,393,76,616,290,782,578,378,611,750,648,465,331,279,546,340,647,218,772,548,334,280,617,746,272,783,587,586,289,409,49,56,457,282,453,68,341,644,610,748,594,674,620,294,564,259,691,397,418,70,212,204,767,625,646,374,88,158,310,499,446,211,217,589,455,405
1,1,189,533,536,535,543,4,743,300,367,299,527,797,798,606,799,115,210,556,81,114,528,306,542,390,304,389,80,3,558,50,559,387,720,684,681,480,483,608,427,101,487,683,482,722,100,184,159,759,98,737,155,225,92,736,163,469,714,226,93,219,349,791,45,408,591,575,660,347,731,11,18,566,421,9,251,711,12,470,574,406,261,37,350,588,432,220,704,811,788,770,762,72,38,400,302,747,276,494,313,491,78,317,383,197,420,807,371,175,193,207,164,60,312,612,369,342,65,590,723,333,291,263,161,244,20,622,504,29,669,58,146,343,64,501,129,619,292,511,283,666,796,702,254,356,143,675,183,776,318,579,695,335,503,668,512,696,214,412,359,153,275,751,550,467,652,130,238,200,521,379,136,199,232,213,618,52,242,771,749,461,396,659,630,192,176,567,713,777,774,336,358,271,119,21,596,27,657,48,509,30,693,410,253,730,281,326,431,364,581,173,110,107,518,90,705,395,285,813,233,122,650,109,549,582,458,325,629,121,613,171,44,265,245,69,761,547,639,627,323,598,42,429,273,638,145,753,460,520,636
2,2,7,572,680,573,682,688,692,191,694,700,766,152,760,151,0,223,430,224,428,113,190,436,758,112,445,120,126,118,236,437,235,439,182,709,727,719,466,658,468,701,181,664,656,362,721,703,357,476,355,474,534,532,539,366,806,127,89,303,301,388,128,91,386,667,475,665,673,477,97,478,392,742,79,35,172,43,541,36,540,735,734,174,180,77,231,229,637,643,41,635,46,188,563,25,252,258,718,26,712,382,234,250,555,133,17,820,141,557,814,19,135,34,710,812,28,209,354,789,607,346,348,794,526,795,519,249,243,170,525,241,517,16,8,10,485,322,111,324,106,108,603,330,595,597,507,502,500,63,144,628,62,508,150,775,781,626,773,510,634,516,142,260,270,269,262,729,160,162,168,728,85,374,646,310,88,158,446,217,499,589,204,70,212,418,691,455,625,405,397,404,571,259,456,59,564,674,620,767,294,87,678,594,447,565,486,498,810,479,497,381,398,205,416,321,748,492,610,426,531,677,341,311,481,453,99,211,419,690,484,493,417,385,196,451,314,454,425,490,370,5

In [11]:
df_kdtree2.to_csv("data/processed/close_bs_data_static_and_dynamic.csv",index=False)